In [21]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [11]:
img_path = 'Image Dataset/Positive'  # Folder containing images
annot_path = 'Image and Annotations/Positive'  # Folder containing annotations

In [ ]:
import os
import shutil
import random
import concurrent.futures

# Define paths
img_path = 'Image Dataset/Positive'
annot_path = 'Image and Annotations/Positive'

# Output folders
output_dir = 'Dataset_Split'
train_img_dir = os.path.join(output_dir, 'train', 'images')
train_annot_dir = os.path.join(output_dir, 'train', 'annotations')
val_img_dir = os.path.join(output_dir, 'validation', 'images')
val_annot_dir = os.path.join(output_dir, 'validation', 'annotations')
test_img_dir = os.path.join(output_dir, 'test', 'images')
test_annot_dir = os.path.join(output_dir, 'test', 'annotations')

# Create directories if not existing
for directory in [train_img_dir, train_annot_dir, val_img_dir, val_annot_dir, test_img_dir, test_annot_dir]:
    os.makedirs(directory, exist_ok=True)

# Load all annotations into a set
annotations = set(os.listdir(annot_path))

# Only add images that have corresponding annotations
data = [(img, img.replace('.jpg', '.txt')) for img in sorted(os.listdir(img_path)) if img.replace('.jpg', '.txt') in annotations]

# Shuffle and split data
random.shuffle(data)
train_split = int(0.7 * len(data))
val_split = int(0.85 * len(data))
train_data, val_data, test_data = data[:train_split], data[train_split:val_split], data[val_split:]

# Define a function to copy files
def copy_file_pair(pair, img_dir, annot_dir):
    img, annot = pair
    shutil.copy(os.path.join(img_path, img), os.path.join(img_dir, img))
    shutil.copy(os.path.join(annot_path, annot), os.path.join(annot_dir, annot))

# Use concurrent copying for speedup
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Copy each dataset split concurrently
    executor.map(lambda pair: copy_file_pair(pair, train_img_dir, train_annot_dir), train_data)
    executor.map(lambda pair: copy_file_pair(pair, val_img_dir, val_annot_dir), val_data)
    executor.map(lambda pair: copy_file_pair(pair, test_img_dir, test_annot_dir), test_data)

print("Data split completed successfully.")


In [ ]:
import os
import cv2
import numpy as np

def bounding_box(canny_edge, image_name, img):
    # Find all points where edges are detected
    edge_points = np.where(canny_edge == 255)
    if len(edge_points[0]) > 0:
        y1, y2 = np.min(edge_points[0]), np.max(edge_points[0])  # Rows: min and max Y coordinates
        x1, x2 = np.min(edge_points[1]), np.max(edge_points[1])  # Columns: min and max X coordinates

        # Normalize bounding box coordinates
        x_center = (x1 + x2) / 2 / img.shape[1]
        y_center = (y1 + y2) / 2 / img.shape[0]
        width = (x2 - x1) / img.shape[1]
        height = (y2 - y1) / img.shape[0]

        file_path = f"file path\\{image_name}.txt" #actual file path differes to maintain privacy
        with open(file_path, 'w') as file:
            file.write(f"0 {x_center} {y_center} {width} {height}")
            print(f"File '{file_path}' created successfully")

# Process images in the 'Positive' folder
dataset_path = "path of datset" #actual file path differes to maintain privacy
for im in os.listdir(dataset_path):
    # if classes == 'Positive':
    #     for image in os.listdir(dataset_path + '/' + classes):
            img_path = dataset_path + '/' + im
            img = cv2.imread(img_path)
            # print(img)
            gray_image_no = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            canny_edge = cv2.Canny(gray_image_no, 200, 255)
            
            # Count non-zero edge pixels
            non_zero_edge_points = np.count_nonzero(canny_edge)
            if non_zero_edge_points >= 65:
                bounding_box(canny_edge, im.replace('.jpg', ''), img)
            # break
